# Visualize environment and custom tasks

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install -r/content/drive/MyDrive/genrl/requirements.txt

In [5]:
import pathlib
import sys
import os
sys.path.append(str(pathlib.Path(os.path.abspath('')).parent))
sys.path.append('/content/drive/MyDrive/genrl/')
sys.path.append('/content/drive/MyDrive/genrl/envs/')
from custom_dmc_tasks import *
from dm_control import suite
import numpy as np

domain = 'stickman'
task = 'sit_knees'

env = suite.load(domain_name=domain, task_name=task, visualize_reward=True)

action_spec = env.action_spec()

# Define a uniform random policy.
def random_policy(time_step):
  del time_step  # Unused.
  return np.random.uniform(low=action_spec.minimum,
                           high=action_spec.maximum,
                           size=action_spec.shape)

def zero_policy(time_step):
  del time_step
  return np.zeros(action_spec.shape)


class GoalSetWrapper:
    def __init__(self, env, goal=None, goal_idx=None):
       self._env = env
       self._env._step_limit = float('inf')
       self._goal = goal
       self._goal_idx = goal_idx

    def step(self, *args, **kwargs):
        if self._goal is not None:
            self.set_goal(self._goal)
        if self._goal_idx is not None:
            self.set_goal_by_idx(self._goal_idx)
        return self._env.step(*args, **kwargs)

    def set_goal_by_idx(self, idx_goal):
        cur = self._env.physics.get_state().copy()
        for idx, goal in idx_goal:
            cur[idx] = goal
        self._env.physics.set_state(cur)
        self._env.step(np.zeros_like(self.action_spec().shape))

    def set_goal(self, goal):
        goal = np.array(goal)
        size = self._env.physics.get_state().shape[0] - goal.shape[0]
        self._env.physics.set_state(np.concatenate((goal, np.zeros([size]))))
        self._env.step(np.zeros_like(self.action_spec().shape))

    def __getattr__(self, name: str):
       return getattr(self._env, name)


env = GoalSetWrapper(env)

/usr/local/lib/python3.11/dist-packages/glfw/__init__.py:916: GLFWError: (65544) b'X11: The DISPLAY environment variable is missing'
  warnings.warn(message, GLFWError)


In [14]:
import matplotlib.pyplot as plt
from envs.custom_dmc_tasks.stickman import StickmanYogaPoses

obs = env.reset()

for _ in range(1):
    env.set_goal(StickmanYogaPoses.sit_knees)

# for _ in range(20):
#     obs = env.step(np.random.randn(*env.action_spec().shape))
print('Rew', obs.reward)

print('Upright', env.physics.torso_upright())
print('Torso height', env.physics.torso_height())

plt.imshow(env.physics.render(camera_id=0))

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


ValueError: A function named 'flipping_custom_v2' already exists in the container and `allow_overriding_keys` is False.

In [7]:
for _ in range(1):
    obs = env.step(np.random.randn(*env.action_spec().shape))

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
env.physics.named.data.qpos

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


FieldIndexer(qpos):
 0          rootz [ 0       ]
 1          rootx [ 0       ]
 2          rooty [ 2.54    ]
 3      right_hip [ 1.53    ]
 4     right_knee [-0.987   ]
 5    right_ankle [-0.31    ]
 6       left_hip [ 0.434   ]
 7      left_knee [-1.26    ]
 8     left_ankle [ 0.252   ]
 9  left_shoulder [ 0.395   ]
10     left_elbow [ 1.73    ]
11 right_shoulder [ 1.16    ]
12    right_elbow [ 2.46    ]

In [9]:
env.physics.named.data.xpos

/usr/local/lib/python3.11/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


FieldIndexer(xpos):
                   x         y         z         
 0         world [ 0         0         0       ]
 1         torso [ 0         0         1.25    ]
 2          neck [ 0.199     0         0.962   ]
 3          head [ 0.256     0         0.88    ]
 4   right_thigh [-0.142    -0.05      1.46    ]
 5     right_leg [-0.751    -0.05      1.32    ]
 6    right_foot [-1.02     -0.05      1.38    ]
 7    left_thigh [-0.142     0.05      1.46    ]
 8      left_leg [-0.475     0.05      1.93    ]
 9     left_foot [-0.479     0.05      2.17    ]
10      left_arm [ 0.114     0.05      1.09    ]
11  left_forearm [-0.198     0.05      1.11    ]
12     left_hand [-0.257     0.05      0.972   ]
13     right_arm [ 0.114    -0.05      1.09    ]
14 right_forearm [-0.0475   -0.05      0.957   ]
15    right_hand [ 0.0854   -0.05      0.887   ]